In [1]:
!pip install datasets
!pip install transformers
# !pip install torch==2.2.2

## Imports

In [2]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
import torch

/home/jdannemi/miniconda3/envs/latent_explr_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
civil_comments = load_dataset("google/civil_comments")

In [15]:
"""
    Functor which adds a column to the input dataset which stores the embeddings
    of the final hidden layer of the given model for each prompt in dataset
"""
class Embeddings_Functor:
    def __init__(self, model_id):
        # Load the model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModel.from_pretrained(model_id)

        # Configure the model to output all its hidden state values
        self.model.output_hidden_states = True
    def __call__(self, dataset_sample):
        # Tokenize the prompts and pass through model
        prompt = dataset_sample["text"]
        model_inputs = self.tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            model_outputs = self.model(**model_inputs)
        
        # Add the embeddings to the dataset
        dataset_sample["embeddings"] = model_outputs.last_hidden_state
        return dataset_sample



In [16]:
## Get the embedding of each prompt and add it to the dataset
hub_model_id = "<HF_MODEL_ID>" 
embedding_functor = Embeddings_Functor(hub_model_id)
# Apply embedding functor to each sample in train to add embeddings column
train_set = civil_comments["train"]
train_set.map(embedding_functor)

Map:   0%|          | 4142/1804874 [02:31<18:14:12, 27.43 examples/s]


KeyboardInterrupt: 

In [5]:
# Save the training set to json
train_set.to_json('train_dataset.csv')

<class 'datasets.arrow_dataset.Dataset'>
